In [5]:
import time
import random
import os
import pandas as pd
import csv
import numpy as np
from itertools import chain
from pdp_lib import processing_google_map as proc
from pdp_lib import util
from GA_lib import GA_multi_depot as GA
from GA_lib import operation_multi_depot as operation
from GA_lib import evaluate_multi_depot as evaluate
from statistics import mode

In [9]:
def solve_and_print(filepath, rounds = 10, population_size = 100, generations=2000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.vote_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,k=3)
    DEPOT_NUMBERS = proc.final_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,k=3)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    print(REQ_BY_DEPOTS)
    print('----------------Instance name: ' + str(filepath) + '-----------------------------------')
    print(" processing time --- %s seconds ---" % (time.time() - process_time))

    total_results = []
    for round in range(rounds):
        results_all_depots = []
        distance_all_depots = []
        time_all_depots = []
        for id_depot in range(len(DEPOTS)):
            THIS_DEP_REQS = REQ_BY_DEPOTS[id_depot]
            NODES_OF_THIS_DEPOT = proc.locations_of_this_depot(id_depot, REQ_BY_DEPOTS, LOCATIONS)
            if(len(NODES_OF_THIS_DEPOT)==0):
                results_all_depots.append([])
                distance_all_depots.append(0)
                time_all_depots.append(0)
                continue
            # ############### SOLVING THE PROBLEMS !!!!!!!! ######################################
            pops_create_time = time.time()
            calculation_time = time.time()
            print('#################### Round :' +str(round+1)+' ############################')
            ## Initialize the populations
            populations = []
            for _ in range(population_size):
                chromosome = GA.initialize_Feasible_chromosome(DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS, LoadCapacities)
                populations.append(chromosome)
            # print("Populations creation time --- %s seconds ---" % (time.time() - pops_create_time))
            ## Crossovers and mutate ##
            GA_time = time.time()

            ## Evaluate fitnesses of all populations ##
            best_fitness_so_far = -99999999
            for gen in range(generations):
                fitness_table = []
                for chromosome in populations:
                    fitness_table.append(
                        evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,
                                                   id_depot))
                populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
                populations.pop()
                populations.pop()
                elite1 = populations.pop(0)
                elite2 = populations.pop(0)


                ## Tournament Selection ##
                # id1, id2 = random.randrange(0, len(populations)), random.randrange(0, len(populations))
                id1,id2,id3,id4 = random.sample(range(0, len(populations)), 4)
                fitness1,fitness2,fitness3,fitness4 = fitness_table[id1],fitness_table[id2],fitness_table[id3],fitness_table[id4]
                candidate1,candidate2,candidate3,candidate4 = populations[id1], populations[id2],populations[id3],populations[id4]
                parent1 = operation.tournament_selection(candidate1,candidate2,fitness1,fitness2)
                parent2 = operation.tournament_selection(candidate3,candidate4,fitness3,fitness4)

                ## Crossing over !! ####
                child1, child2 = operation.crossover(DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, parent1, parent2,
                                                     DEMANDS, LoadCapacities, DISTANCES_FROM_DEPOTS,
                                                     DISTANCES_TO_DEPOTS, id_depot, crossoverRate)
                # if(not evaluate.haveEqualNodes(child1,child2,LOCATIONS)):
                #         print('note have Equal nodes, Crossover Bug!!!!!'+ str(gen))
                #         break
                #
                ## Mutation ##
                child1 = operation.mutate(child1, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                child2 = operation.mutate(child2, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                if (not evaluate.haveEqualNodes(child1, child2, NODES_OF_THIS_DEPOT)):
                    print('note have Equal nodes, Mutation Bug!!!!!' + str(gen))
                    break


                ## Evaluate Fitness of the best ones ###
                current_fitness = evaluate.chromosomeFitness(child1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                             DISTANCES_TO_DEPOTS, id_depot)
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(child2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))

                ## Add them back into populations ###
                populations.append(child1)
                populations.append(child2)
                populations.append(elite1)
                populations.append(elite2)

                ### Check break conditions ####
                if (current_fitness > best_fitness_so_far):
                    best_fitness_so_far = current_fitness
                    bestFitGen = gen
                    # print('#### New Best Fitness !! , Best so far is :' + str(10000.0 / best_fitness_so_far) + '#####')
                    # print('#### This Generation: ' + str(gen) + '#######')
                if (gen - bestFitGen >= 1000):
                    # print('#### Break Generation: ' + str(gen) + '#######')
                    break
                # print('############# Round-'+str(round+1) +',DEPOT-'+str(id_depot)+','+ 'Generation:' + str(gen + 1) + ' #########################')
                # print('This Gen Distance :' + str(10000.0 / current_fitness))

            ### Evaluate results of current DEPOT ####
            fitness_table = []
            for chromosome in populations:
                fitness_table.append(
                    evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot))
            populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
            # print("GA time --- %s seconds ---" % (time.time() - GA_time))
            best_chromosome = populations[0]
            computational_time = time.time() - calculation_time
            # print("Total Calculation time --- %s seconds ---" % (computational_time))
            distance = evaluate.chromosomeRoutesDistance(best_chromosome,DISTANCES,DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,id_depot)
            # print('Distances of the best chromosome: ' + str(distance))
            # print(best_chromosome)

            ## Memo the results ##
            results_all_depots.append(best_chromosome)
            distance_all_depots.append(distance)
            time_all_depots.append(computational_time)
        total_results.append((results_all_depots,distance_all_depots,time_all_depots))
        print(results_all_depots)
    return total_results

In [11]:
def mean_results(results):
    solutions = []
    cal_times = []
    distances = []
    for round, round_result in enumerate(results):
        sol = results[round][0]
        dist = results[round][1]
        ct = results[round][2]

        solutions.append(sol)
        distances.append(dist)
        cal_times.append(ct)
    distances = np.array(distances)
    distances = pd.DataFrame(distances)
    distances = np.mean(distances, axis=0)
    cal_times = np.array(cal_times)
    cal_times = pd.DataFrame(cal_times)
    cal_times = np.mean(cal_times, axis=0)
    return solutions,distances, cal_times


In [12]:
def create_worst_case(filepath, rounds = 1, population_size = 100, generations=3000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    
    res = GA.initialize_WorstCase_Chromosome(REQUESTS)
    
    
    dist_worst = 0
    for [idx,_,[p,d]] in res:
        depot = int(DEPOT_NUMBERS[idx]) 
        dist_worst += (DISTANCES_FROM_DEPOTS[depot][p]+DISTANCES[p][d]+DISTANCES_TO_DEPOTS[d][depot])

    return res,dist_worst

# Automated run

In [25]:
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))

[{13: (31, 48), 15: (35, 49), 19: (41, 34)}, {38: (79, 61)}, {0: (3, 66), 1: (6, 59), 2: (7, 33), 3: (9, 12), 4: (15, 43), 5: (16, 11), 6: (18, 42), 7: (19, 53), 8: (24, 2), 9: (26, 68), 10: (28, 71), 11: (29, 69), 12: (30, 23), 14: (32, 4), 16: (37, 22), 17: (39, 72), 18: (40, 13), 20: (44, 78), 21: (45, 25), 22: (46, 70), 23: (47, 50), 24: (51, 5), 25: (52, 21), 26: (54, 36), 27: (55, 27), 28: (56, 38), 29: (57, 1), 31: (62, 20), 32: (63, 77), 33: (64, 65), 34: (67, 14), 36: (75, 17), 37: (76, 8), 39: (80, 10)}, {30: (60, 58)}, {35: (74, 73)}]
----------------Instance name: pdp_instances/GOO/doing/goo1.csv-----------------------------------
 processing time --- 0.43802452087402344 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


...

#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


[[[1, [13, 15, 19], [35, 49, 31, 41, 48, 34]], [0, [], []]], [[0, [38], [79, 61]]], [[3, [37, 24, 36], [51, 76, 75, 5, 17, 8]], [4, [39, 4, 31, 23, 12], [15, 80, 62, 43, 47, 30, 23, 20, 10, 50]], [1, [22, 2, 33, 11, 20], [7, 29, 33, 46, 44, 70, 78, 69, 64, 65]], [0, [8, 16, 18, 3, 21], [9, 24, 12, 2, 40, 45, 37, 25, 22, 13]], [6, [32, 27, 6, 26, 17, 10], [63, 54, 55, 77, 18, 36, 39, 27, 28, 42, 71, 72]], [5, [1, 29, 25, 28, 7], [56, 57, 52, 38, 1, 6, 21, 19, 59, 53]], [7, [34, 9, 0, 14, 5], [32, 26, 3, 16, 4, 68, 67, 66, 14, 11]]], [[0, [30], [60, 58]]], [[0, [35], [74, 73]]]]


# The GOO Instances

# Average list by columns

In [27]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [28]:
# res =results[0]
# SOL,DIST,CT = res[0],res[1],res[2]


In [29]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [30]:
# worst_sol,distance_worst = create_worst_case('pdp_instances/GOO/goo12.csv')
# distance_worst

In [32]:
# SOL[2]
# count_num_vehicle([worst_sol])

# Writing to CSV

In [34]:
solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo12.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo12.csv')

# Average list by columns

In [35]:

# worst_sol

In [36]:
def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

In [37]:
# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_goo.csv')

[1.0, 1.0, 7.5, 1.0, 1.0]


In [38]:
# results = [([[[2,[16,19],[41,37,34,22]],[1,[13,4,6],[15,31,43,18,48,42]],[0,[15],[35,49]]],[[1,[39,25,24],[52,51,80,21,5,10]],[0,[38],[79,61]]],[[2,[23,7,21],[19,47,45,50,53,25]],[5,[12,31,18],[62,40,30,23,20,13]],[3,[2,32,14,11,27,10,33],[32,7,4,29,33,63,55,77,69,28,27,64,71,65]],[1,[],[]],[0,[34,9,8,0],[3,26,24,68,67,66,2,14]],[4,[3,5],[9,16,12,11]]],[[0,[30],[60,58]]],[[3,[37,35,36],[74,75,73,76,17,8]],[2,[],[]],[1,[29,28,1],[56,57,38,1,6,59]],
                                                                                                                                                                                                                                                                                                                                                                                       [0,[26,22,20,17],[54,46,44,36,39,70,78,72]]]])]

IndentationError: unexpected indent (<ipython-input-38-06f82f746798>, line 2)

In [19]:
# res = results[0]

In [20]:
# all_results

In [24]:
# 
# num_vehicles = np.array(num_vehicles)
# df_num_vehicles = pd.DataFrame(num_vehicles)
# df_num_vehicles.to_csv('num_veh_goo.csv')

In [79]:
c =set(c)

In [80]:
temp = [{8: (22, 49), 11: (29, 59), 12: (31, 18), 13: (33, 30), 14: (36, 9), 15: (38, 11), 16: (39, 14), 19: (43, 37), 20: (44, 32), 21: (45, 34), 26: (57, 47), 29: (62, 80)}, {22: (48, 5), 24: (55, 26), 28: (61, 7), 39: (79, 53)}, {0: (1, 2), 1: (3, 35), 2: (4, 64), 3: (8, 13), 5: (12, 24), 7: (20, 17), 9: (23, 41), 10: (25, 16), 30: (63, 50), 31: (65, 21), 32: (66, 46), 35: (70, 6), 36: (72, 28)}, {27: (60, 77), 34: (69, 58)}, {4: (10, 54), 6: (19, 71), 17: (40, 74), 18: (42, 67), 23: (51, 75), 25: (56, 27), 33: (68, 73), 37: (76, 52), 38: (78, 15)}]

In [81]:
temp[2]
s = set([x for x in temp[2].values()])
s

{(1, 2),
 (3, 35),
 (4, 64),
 (8, 13),
 (12, 24),
 (20, 17),
 (23, 41),
 (25, 16),
 (63, 50),
 (65, 21),
 (66, 46),
 (70, 6),
 (72, 28)}

In [82]:
worst_sol2 = [ x for x in worst_sol if tuple(x[2]) in s]

In [83]:
worst_sol2

[]